In [8]:
import os
import sys
import glob

try:
    sys.path.append(glob.glob('carla/PythonAPI/carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import hydra

from omegaconf import DictConfig, OmegaConf
from hydra.core.config_store import ConfigStore

from utils.weather import Weather
from schemas.pgm_schema import PGMModel
from schemas.weather_schema import WeatherSchema, SunSchema

In [11]:
cs = ConfigStore.instance()
cs.store(group="weather", name="base_weather_model", node=WeatherSchema)
cs.store(group="weather/sun", name="base_sun_model", node=SunSchema)
cs.store(group="model", name="base_pgm_model", node=PGMModel)
with hydra.initialize(version_base=None, config_path="conf"):
    cfg = hydra.compose(
        config_name="config", 
        overrides=[
            "weather=clear",
            "weather/sun=day"
            
            ])
    print(OmegaConf.to_yaml(cfg.weather))

cloudiness: 10.0
rain: 0.0
puddles: 0.0
wind: 5.0
fog: 0.0
wetness: 0.0
sun:
  azimuth: 0.0
  altitude: 45.0



In [10]:
client = carla.Client(cfg.host, cfg.port)
client.set_timeout(10.0)
world = client.get_world()

weather = Weather(world.get_weather())
weather.setWeather(cfg.weather)
print(weather)
world.set_weather(weather.weather)

Sun(alt: 45.00, azm: 0.00) Storm(clouds=0%, rain=0%, wind=0%)
